In [1]:
#Import Libraries
import pandas as pd
import plotly.express as px
from pathlib import Path
import numpy as np
import re

In [2]:
# Load CSVs
books_path = Path('Resources/books.csv')
ratings_path = Path('Resources/ratings.csv')
users_path = Path('Resources/users.csv')

In [3]:
# Check CSV encoding
with open(books_path) as f:
    print(f)

<_io.TextIOWrapper name='Resources/books.csv' mode='r' encoding='UTF-8'>


In [4]:
# Read CSV with latin-1 encoding due to variety of latin characters
# And assign data types for consistency
# Columns ISBN and Year-Of-Publication cannot be read in as int data types. For example, ISBN contains alphanumeric values. Year-Of-Publication column contains alphabetical values and not exclusively number values. This will be managed below

books_df = pd.read_csv(
    books_path,
    sep=';',
    on_bad_lines='warn',
    encoding='latin-1', 
    dtype={
        'ISBN': str,
        'Book-Title': str,
        'Book-Author': str,
        'Year-Of-Publication': str,
        'Image-URL-S': str,
        'Image-URL-M': str,
        'Image-URL-L': str,
        }
    )

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


### Note: The warning above indicates rows with inconsistent columns to the rest of the dataset and are omitted from the cleaned DataFrame

In [5]:
# Read CSV
ratings_df = pd.read_csv(ratings_path, sep=';', on_bad_lines='warn', encoding='latin-1')

In [6]:
# Read CSV
users_df = pd.read_csv(users_path, sep=';', on_bad_lines='warn', encoding='latin-1')

In [7]:
# Check head
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
#Check columns
books_df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [9]:
# Check head
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [10]:
# Check head
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
# Check shape
print('books_df', books_df.shape)
print('ratings_df', ratings_df.shape)
print('users_df', users_df.shape)

books_df (271360, 8)
ratings_df (1149780, 3)
users_df (278858, 3)


In [12]:
# Check nulls
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [13]:
# Check nulls
ratings_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [14]:
# Check nulls
users_df.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [15]:
# Define function to locate index of values (Source: https://www.geeksforgeeks.org/find-location-of-an-element-in-pandas-dataframe-in-python/)
def get_indexes(object, value):
     
    # Empty list
    position_list = []
     
    # isin() method will return a DataFrame with
    # boolean values, True at the positions   
    # where element exists
    result = object.isin([value])
     
    # any() method will return
    # a boolean series
    series_object = result.any()
 
    # Get list of column names where
    # element exists
    column_names = list(series_object[series_object == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in column_names:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            position_list.append((row, col))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return position_list

In [16]:
# Apply getIndexes function to locate null values
positions = get_indexes(books_df, np.nan)
print('Index positions of NaN in books_df: ')
for i in range (len(positions)):
    print(positions[i])

Index positions of NaN in books_df: 
(187689, 'Book-Author')
(128890, 'Publisher')
(129037, 'Publisher')
(209538, 'Image-URL-L')
(220731, 'Image-URL-L')
(221678, 'Image-URL-L')


# Manage Null Values

## Books DataFrame

In [17]:
# Check NaN to correct missing values
books_df.iloc[187689]

ISBN                                                          9627982032
Book-Title             The Credit Suisse Guide to Managing Your Perso...
Book-Author                                                          NaN
Year-Of-Publication                                                 1995
Publisher                                 Edinburgh Financial Publishing
Image-URL-S            http://images.amazon.com/images/P/9627982032.0...
Image-URL-M            http://images.amazon.com/images/P/9627982032.0...
Image-URL-L            http://images.amazon.com/images/P/9627982032.0...
Name: 187689, dtype: object

In [18]:
# Check NaN to correct missing values
books_df.iloc[128890]

ISBN                                                          193169656X
Book-Title                                                   Tyrant Moon
Book-Author                                              Elaine Corvidae
Year-Of-Publication                                                 2002
Publisher                                                            NaN
Image-URL-S            http://images.amazon.com/images/P/193169656X.0...
Image-URL-M            http://images.amazon.com/images/P/193169656X.0...
Image-URL-L            http://images.amazon.com/images/P/193169656X.0...
Name: 128890, dtype: object

In [19]:
# Check NaN to correct missing values
books_df.iloc[129037]

ISBN                                                          1931696993
Book-Title                                               Finders Keepers
Book-Author                                              Linnea Sinclair
Year-Of-Publication                                                 2001
Publisher                                                            NaN
Image-URL-S            http://images.amazon.com/images/P/1931696993.0...
Image-URL-M            http://images.amazon.com/images/P/1931696993.0...
Image-URL-L            http://images.amazon.com/images/P/1931696993.0...
Name: 129037, dtype: object

In [20]:
# Check NaN to correct missing values
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: Creating the X-Men, How It All Beg...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/078946697X.0...
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L                                                          NaN
Name: 209538, dtype: object

In [21]:
# Check NaN to correct missing values
books_df.iloc[220731]

ISBN                                                          2070426769
Book-Title             Peuple du ciel, suivi de 'Les Bergers\";Jean-M...
Book-Author                                                         2003
Year-Of-Publication                                            Gallimard
Publisher              http://images.amazon.com/images/P/2070426769.0...
Image-URL-S            http://images.amazon.com/images/P/2070426769.0...
Image-URL-M            http://images.amazon.com/images/P/2070426769.0...
Image-URL-L                                                          NaN
Name: 220731, dtype: object

In [22]:
# Check NaN to correct missing values
books_df.iloc[221678]

ISBN                                                          0789466953
Book-Title             DK Readers: Creating the X-Men, How Comic Book...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/0789466953.0...
Image-URL-S            http://images.amazon.com/images/P/0789466953.0...
Image-URL-M            http://images.amazon.com/images/P/0789466953.0...
Image-URL-L                                                          NaN
Name: 221678, dtype: object

In [23]:
# Add missing author and publisher values as per research on Amazon and bookfinder.com via ISBN search
books_df.at[187689, 'Book-Author'] = 'Larissa Anne Downes, Editor'
books_df.at[128890, 'Publisher'] = 'CreateSpace Independent Publishing Platform'
books_df.at[129037, 'Publisher'] = 'NovelBooks Inc'

In [25]:
# Check corrected value
books_df.iloc[187689]

ISBN                                                          9627982032
Book-Title             The Credit Suisse Guide to Managing Your Perso...
Book-Author                                  Larissa Anne Downes, Editor
Year-Of-Publication                                                 1995
Publisher                                 Edinburgh Financial Publishing
Image-URL-S            http://images.amazon.com/images/P/9627982032.0...
Image-URL-M            http://images.amazon.com/images/P/9627982032.0...
Image-URL-L            http://images.amazon.com/images/P/9627982032.0...
Name: 187689, dtype: object

In [26]:
# Check corrected value
books_df.iloc[6452]

ISBN                                                          0307121259
Book-Title                             I Am a Bunny (Golden Sturdy Book)
Book-Author                                                     O. Risom
Year-Of-Publication                                                 1963
Publisher                                                   Golden Books
Image-URL-S            http://images.amazon.com/images/P/0307121259.0...
Image-URL-M            http://images.amazon.com/images/P/0307121259.0...
Image-URL-L            http://images.amazon.com/images/P/0307121259.0...
Name: 6452, dtype: object

In [29]:
# Check NaN to correct missing value
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: Creating the X-Men, How It All Beg...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/078946697X.0...
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L                                                          NaN
Name: 209538, dtype: object

In [30]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[209538] = books_df.iloc[209538].shift()
books_df.at[209538, 'ISBN'] = '078946697X'
books_df.at[209538, 'Book-Title'] = 'DK Readers: The Story of the X-Men, How It All Began (Level 4: Proficient Readers)'
books_df.at[209538, 'Book-Author'] = 'Michael Teitelbaum'
books_df.at[209538, 'Publisher'] = 'DK Publishing Inc'
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: The Story of the X-Men, How It All...
Book-Author                                           Michael Teitelbaum
Year-Of-Publication                                                 2000
Publisher                                              DK Publishing Inc
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L            http://images.amazon.com/images/P/078946697X.0...
Name: 209538, dtype: object

In [31]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[220731] = books_df.iloc[220731].shift()
books_df.at[220731, 'ISBN'] = '2070426769'
books_df.at[220731, 'Book-Title'] = 'Peuple du ciel suivi de les Bergers'
books_df.at[220731, 'Book-Author'] = 'Jean-Marie Gustave'
books_df.at[220731, 'Publisher'] = 'Gallimard'
books_df.iloc[220731]

ISBN                                                          2070426769
Book-Title                           Peuple du ciel suivi de les Bergers
Book-Author                                           Jean-Marie Gustave
Year-Of-Publication                                                 2003
Publisher                                                      Gallimard
Image-URL-S            http://images.amazon.com/images/P/2070426769.0...
Image-URL-M            http://images.amazon.com/images/P/2070426769.0...
Image-URL-L            http://images.amazon.com/images/P/2070426769.0...
Name: 220731, dtype: object

In [32]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[221678] = books_df.iloc[221678].shift()
books_df.at[221678, 'ISBN'] = '2070426769'
books_df.at[221678, 'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books_df.at[221678, 'Book-Author'] = 'James Buckley'
books_df.at[221678, 'Publisher'] = 'DK Publishing Inc'
books_df.iloc[221678]

ISBN                                                          2070426769
Book-Title             DK Readers: Creating the X-Men, How Comic Book...
Book-Author                                                James Buckley
Year-Of-Publication                                                 2000
Publisher                                              DK Publishing Inc
Image-URL-S            http://images.amazon.com/images/P/0789466953.0...
Image-URL-M            http://images.amazon.com/images/P/0789466953.0...
Image-URL-L            http://images.amazon.com/images/P/0789466953.0...
Name: 221678, dtype: object

In [33]:
# Define function to detect alphanumeric values
# Source - https://www.geeksforgeeks.org/how-to-check-string-is-alphanumeric-or-not-using-regular-expression/

# Function to check string
# is alphanumeric or not
def isAlphaNumeric(str):

	# Regex to check string is
	# alphanumeric or not.
	regex = "^(?=.*[a-zA-Z])(?=.*[0-9])[A-Za-z0-9]+$"

	# Compile the ReGex
	p = re.compile(regex)

	# If the string is empty
	# return false
	if(str == None):
		return False

	# Return if the string
	# matched the ReGex
	if(re.search(p, str)):
		return True
	else:
		return False


In [34]:
# Store ISBN values to list
ISBN_list = books_df['ISBN'].values.tolist()

In [35]:
# List comprehension for Alphanumeric check on ISBN_List
alphanumeric_list = [isAlphaNumeric(i) for i in ISBN_list]

In [36]:
# Count for True
alphanumeric_list.count(True)

22330

In [37]:
# Count for False
alphanumeric_list.count(False)

249030

In [38]:
# Store Year-Of-Publication to list
pub_year_list = books_df['Year-Of-Publication'].values.tolist()

In [39]:
# List comprehension for alphanumeric check on ISBN_List
alphanumeric_pub_list = [isAlphaNumeric(i) for i in pub_year_list]

In [40]:
# Check Count for True
alphanumeric_pub_list.count(True)

0

In [41]:
#Check count for False
alphanumeric_pub_list.count(False)

271360

In [42]:
# Check for alpha in Year-of-Publication column
print(any(i.isalpha() for i in pub_year_list))

False


In [31]:
# Drop Image-URL columns since they do not add value for recommendation system
books_df.drop(books_df.iloc[:, 5:8], inplace=True, axis=1)

In [32]:
# Check head
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [33]:
# Write cleaned DataFrame to csv
# books_df.to_csv('Resources/books_cleaned.csv')

## Manage Nulls - Users DataFrame

# Run ML for books recommendations where user age values are available
# Run ML for books recommendations for all users. Compare results

In [45]:
# Check min
users_df['Age'].min()

0.0

In [46]:
# Check Age min count
users_df['Age'].count().min()

168096

In [48]:
# Fill Age column NA values with -1 to cast data type to int -
# -1 chosen since there are pre-existing 0
users_df['Age'].fillna(-1, inplace=True)

In [49]:
# Convert Age colum data type to int
users_df['Age'] = users_df['Age'].astype(int)

In [50]:
# Check head
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",-1
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",-1
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",-1


In [34]:
# Write cleaned DataFrame to csv
users_df.to_csv('Resources/users_cleaned.csv')